# Fetch the raw data from the mutall_rental database

## 1. Write a query to retrieve the start of the reporting preiod

In [1]:
#
#Compose the report period year given a month and the year
from datetime import date
#
#Define the year and  month 
year = 2023
month = 9
#
#Compose the date
current_reporting = date(year, month, 1)
#
#Subtract one month from the current date to get the previous reporting period
previous_reporting = current_reporting.replace(day=1, month=month-1)
#
#Using the two dates above write an sql to get the start of the report
#period from the period table in the mutal rental database
reporting_start_sql = """
SELECT 
    if(
        cutoff is null,
        %s,
        cutoff
    ) as `start`
FROM 
    period
WHERE 
    month = '%s' AND year = '%s'
""" % (current_reporting, previous_reporting.month, previous_reporting.year)

## 2.Use the queries to get the raw data form the database into a dataframe

In [2]:
#
# This library allows Python to connect and interact with MySQL databases.
import mysql.connector
#
#This library is used for data manipulation and analysis.
#It provides data structures like DataFrames which are convenient for handling tabular data
import pandas as pd
#
#Establish a connection to a MySQL database.  
connection = mysql.connector.connect(
    #
    #The location of the database server.
    host="localhost", 
    #
    #The port number used by the MySQL server
    port=3306,
    #
    #The username for accessing the database.
    user="mutall",
    #
    #The password for the user
    passwd="mutall@2023",
    #
    #The name of the specific database you to connect to.
    db="mutall_rental"
  )
#
#A cursor is used to interact with the database, executing queries and fetching results.
#The dictionary=True option specifies that the results should be returned as dictionaries,
#making it easier to access data by column names.  
cursor = connection.cursor(dictionary=True) 
#
#The base sql that retrieves the raw data
#TODO: Fetch all the raw data !!
#This is the query that retrieves agreement data
agreement_sql = 'select * from agreement'
#
#This is the query that retrives client payment method (Quaterly or monthly)
client_sql = 'select client,quarterly from client'
#
#Executes the SQL statements in the content variable    
cursor.execute(agreement_sql)
#
#Retrieves the results of the executed SQL statements and stores them in 
#a variable named result. The cursor.fetchall() method fetches all rows from the result set.
agreement = cursor.fetchall()
#
#Execute the client sql
cursor.execute(client_sql)
#
#Fetch the client results
client = cursor.fetchall()
#
#Execute the reporting period start date query
cursor.execute(reporting_start_sql)
#
#Fetch all the results gotten from the reporting period
reporting_start = cursor.fetchall()
#
#Convert the clients data to a dataframe
client_df = pd.DataFrame(data=client)
#
#Converts the agreement data to a dataframe
agreement_df = pd.DataFrame(data=agreement)
#
#Merge the two dataframes
df = agreement_df.merge(client_df, how='inner', on="client")
#
#Add the start_date of the reporting period to the dataframe
df['rpt_start'] = reporting_start[0]['start']
#
#Close the connection even if errors occur  
connection.close() 
df

,agreement,room,client,amount,start_date,duration,review,terminated,valid,comment,quarterly,rpt_start
0,1,24,1,9500.0,2014-04-01,5.0,3,2020-02-29,1,Micro-finance.,1,2023-08-31
1,2,5,2,5500.0,2017-10-01,5.0,3,None,1,Bar.,1,2023-08-31
2,3,6,2,47880.0,2012-04-01,5.0,3,None,1,Bar.,1,2023-08-31
3,4,31,2,5000.0,2013-10-01,5.0,3,None,1,Bar.,1,2023-08-31
4,5,32,2,5000.0,2013-10-01,5.0,3,None,1,Bar.,1,2023-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...
103,119,75,71,18000.0,2023-12-31,3.0,3,2024-01-22,0,Terminated Agreement.Kentagon Room set to star...,0,2023-08-31
104,124,75,71,18000.0,2024-01-15,3.0,3,None,0,Terminated for Improper Accounting records whi...,0,2023-08-31
105,123,7,72,22000.0,2023-12-31,3.0,3,None,1,Agreement for lease between Royal Nest propert...,0,2023-08-31
106,125,75,73,18000.0,2024-01-15,3.0,3,None,1,Agreement between Think Twice Kiserian and Mut...,1,2023-08-31


In [3]:
#
#Get summary statistics on your dataset
df.describe().round(1)

,agreement,room,client,amount,duration,review,valid,quarterly
count,108.0,108.0,108.0,94.0,105.0,108.0,108.0,108.0
mean,61.0,30.4,32.6,28455.2,4.6,3.0,0.9,0.5
std,37.0,22.6,20.5,42482.2,0.9,0.6,0.2,0.5
min,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
25%,27.8,11.0,17.8,10000.0,5.0,3.0,1.0,0.0
50%,61.5,22.0,33.5,17000.0,5.0,3.0,1.0,0.5
75%,92.2,48.2,43.0,22000.0,5.0,3.0,1.0,1.0
max,126.0,75.0,74.0,206161.2,5.0,5.0,1.0,1.0


# 3.Cleaning and preprocessing the data

In [4]:
#
#Remove unnecessary column   
df.drop('comment', axis=1, inplace=True)
df

,agreement,room,client,amount,start_date,duration,review,terminated,valid,quarterly,rpt_start
0,1,24,1,9500.0,2014-04-01,5.0,3,2020-02-29,1,1,2023-08-31
1,2,5,2,5500.0,2017-10-01,5.0,3,None,1,1,2023-08-31
2,3,6,2,47880.0,2012-04-01,5.0,3,None,1,1,2023-08-31
3,4,31,2,5000.0,2013-10-01,5.0,3,None,1,1,2023-08-31
4,5,32,2,5000.0,2013-10-01,5.0,3,None,1,1,2023-08-31
...,...,...,...,...,...,...,...,...,...,...,...
103,119,75,71,18000.0,2023-12-31,3.0,3,2024-01-22,0,0,2023-08-31
104,124,75,71,18000.0,2024-01-15,3.0,3,None,0,0,2023-08-31
105,123,7,72,22000.0,2023-12-31,3.0,3,None,1,0,2023-08-31
106,125,75,73,18000.0,2024-01-15,3.0,3,None,1,1,2023-08-31


In [5]:
#
#Filter to get valid and active agreements
df = df[df['valid'] == 1 & df['terminated'].isnull()]
df

,agreement,room,client,amount,start_date,duration,review,terminated,valid,quarterly,rpt_start
1,2,5,2,5500.0,2017-10-01,5.0,3,None,1,1,2023-08-31
2,3,6,2,47880.0,2012-04-01,5.0,3,None,1,1,2023-08-31
3,4,31,2,5000.0,2013-10-01,5.0,3,None,1,1,2023-08-31
4,5,32,2,5000.0,2013-10-01,5.0,3,None,1,1,2023-08-31
5,6,33,2,5000.0,2013-10-01,5.0,3,None,1,1,2023-08-31
6,7,34,2,5000.0,2013-10-01,5.0,3,None,1,1,2023-08-31
7,8,36,2,5000.0,2013-10-01,5.0,3,None,1,1,2023-08-31
8,9,37,2,5000.0,2013-10-01,5.0,3,None,1,1,2023-08-31
9,37,46,2,NaN,2013-10-01,5.0,3,None,1,1,2023-08-31
10,39,51,2,5000.0,2013-10-01,5.0,3,None,1,1,2023-08-31


In [6]:
#
#Drop the terminated and valud columns since going foward we are not going to use them
df = df.drop(['terminated', 'valid'], axis=1)
df

,agreement,room,client,amount,start_date,duration,review,quarterly,rpt_start
1,2,5,2,5500.0,2017-10-01,5.0,3,1,2023-08-31
2,3,6,2,47880.0,2012-04-01,5.0,3,1,2023-08-31
3,4,31,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
4,5,32,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
5,6,33,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
6,7,34,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
7,8,36,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
8,9,37,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
9,37,46,2,NaN,2013-10-01,5.0,3,1,2023-08-31
10,39,51,2,5000.0,2013-10-01,5.0,3,1,2023-08-31


In [7]:
df.columns
df

,agreement,room,client,amount,start_date,duration,review,quarterly,rpt_start
1,2,5,2,5500.0,2017-10-01,5.0,3,1,2023-08-31
2,3,6,2,47880.0,2012-04-01,5.0,3,1,2023-08-31
3,4,31,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
4,5,32,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
5,6,33,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
6,7,34,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
7,8,36,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
8,9,37,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
9,37,46,2,NaN,2013-10-01,5.0,3,1,2023-08-31
10,39,51,2,5000.0,2013-10-01,5.0,3,1,2023-08-31


In [8]:
df.dtypes

agreement       int64
room            int64
client          int64
amount        float64
start_date     object
duration      float64
review          int64
quarterly       int64
rpt_start      object
dtype: object

In [9]:
#
#Subtract the start date form the rpt_start and get the difference in months
#
#Add a factor column that will be used in conjunction with the unit amount ot 
#establish how much a client is supposed to pay in the given month
#
#If a user pays monthly the factor will always be one
#
#If a user pays quarterly the factor could either be a 0 if the quater is not 
#yet over or a 4 if the quater is over and the cilent is due
 

In [10]:
#
#Assuming columns 'start_date' and 'rpt_start' contain date strings
#convert to datetime
df['rpt_start'] = pd.to_datetime(df['rpt_start'])
df['start_date'] = pd.to_datetime(df['start_date'])
df

,agreement,room,client,amount,start_date,duration,review,quarterly,rpt_start
1,2,5,2,5500.0,2017-10-01,5.0,3,1,2023-08-31
2,3,6,2,47880.0,2012-04-01,5.0,3,1,2023-08-31
3,4,31,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
4,5,32,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
5,6,33,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
6,7,34,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
7,8,36,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
8,9,37,2,5000.0,2013-10-01,5.0,3,1,2023-08-31
9,37,46,2,NaN,2013-10-01,5.0,3,1,2023-08-31
10,39,51,2,5000.0,2013-10-01,5.0,3,1,2023-08-31


## 4.Calculate a revised amount based on the phase

In [11]:
#
#Calculate the duration of the contract in years

#
#Classify the contract into various phases based on the duration :-
#If a contract age is below the review period then the contract is in the normal phase
#If the contract age is between the review period and the renew  period then the contract is in the 
#review phase
#Otherwise the contract is in the renew period
#

#
#Using the phase classification above determine wether the rent amount should change or not 
#and if it should change by what percentage

In [12]:
#
#This library is use for numerical analysis 
import numpy as np
#
#Calculate the duration of the contract in years
df['age'] = (df['rpt_start'] - df['start_date']) / np.timedelta64(1, 'Y')
#
# Convert to whole years
df['age'] = df['age'].astype(int)
#
#Filter out all the negative values
df = df[df['age'] >= 0] 
#
# Classify contract phase based on age
conditions = [df['age'] < df['review'],  # Condition 1
              (df['review'] <= df['age']) & (df['age'] < df['duration'])]  # Condition 2
# 
# Values to assign based on conditions
values = ["normal", "review"]
#
# Assign values
df['phase'] = np.select(conditions, values, default="renewal")
df

,agreement,room,client,amount,start_date,duration,review,quarterly,rpt_start,age,phase
1,2,5,2,5500.0,2017-10-01,5.0,3,1,2023-08-31,5,renewal
2,3,6,2,47880.0,2012-04-01,5.0,3,1,2023-08-31,11,renewal
3,4,31,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal
4,5,32,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal
5,6,33,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal
6,7,34,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal
7,8,36,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal
8,9,37,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal
9,37,46,2,NaN,2013-10-01,5.0,3,1,2023-08-31,9,renewal
10,39,51,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal


In [13]:
#
#Set of possible conditions
conditions = [df['phase'] == 'review', df['phase'] == 'renewal']
#
# Values to assign based on conditions
values = [df['amount'] * 1.1, df['amount'] * (1.2 ** (df['age'] - df['duration']))] 
#
# Assign values
df['revised_amount'] = np.select(conditions, values, default = df['amount'])  
df

,agreement,room,client,amount,start_date,duration,review,quarterly,rpt_start,age,phase,revised_amount
1,2,5,2,5500.0,2017-10-01,5.0,3,1,2023-08-31,5,renewal,5500.00000
2,3,6,2,47880.0,2012-04-01,5.0,3,1,2023-08-31,11,renewal,142968.91392
3,4,31,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000
4,5,32,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000
5,6,33,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000
6,7,34,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000
7,8,36,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000
8,9,37,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000
9,37,46,2,NaN,2013-10-01,5.0,3,1,2023-08-31,9,renewal,NaN
10,39,51,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000


In [14]:
df.to_csv('output.csv', index=False)

In [15]:
df['month_diff'] = (df['rpt_start'] - df['start_date']) / np.timedelta64(1, 'M')
df['month_diff'] = df['month_diff'].astype(int)  # Convert to whole months
df


,agreement,room,client,amount,start_date,duration,review,quarterly,rpt_start,age,phase,revised_amount,month_diff
1,2,5,2,5500.0,2017-10-01,5.0,3,1,2023-08-31,5,renewal,5500.00000,70
2,3,6,2,47880.0,2012-04-01,5.0,3,1,2023-08-31,11,renewal,142968.91392,136
3,4,31,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118
4,5,32,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118
5,6,33,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118
6,7,34,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118
7,8,36,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118
8,9,37,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118
9,37,46,2,NaN,2013-10-01,5.0,3,1,2023-08-31,9,renewal,NaN,118
10,39,51,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118


In [16]:
#
#Do a modulus operation to get all quarterly clients who are due
df['mod'] = df['month_diff'] % 3
#
#Print the dataframe
df

,agreement,room,client,amount,start_date,duration,review,quarterly,rpt_start,age,phase,revised_amount,month_diff,mod
1,2,5,2,5500.0,2017-10-01,5.0,3,1,2023-08-31,5,renewal,5500.00000,70,1
2,3,6,2,47880.0,2012-04-01,5.0,3,1,2023-08-31,11,renewal,142968.91392,136,1
3,4,31,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1
4,5,32,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1
5,6,33,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1
6,7,34,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1
7,8,36,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1
8,9,37,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1
9,37,46,2,NaN,2013-10-01,5.0,3,1,2023-08-31,9,renewal,NaN,118,1
10,39,51,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1


In [17]:
#
#Add a factor column that will be used in conjunction with the unit amount ot 
#establish how much a client is supposed to pay in the given month
df['factor'] = np.where((df['quarterly'] == 0), 1, (df['mod'] == 0) * 3)
#
#Print the dataframe
df

,agreement,room,client,amount,start_date,duration,review,quarterly,rpt_start,age,phase,revised_amount,month_diff,mod,factor
1,2,5,2,5500.0,2017-10-01,5.0,3,1,2023-08-31,5,renewal,5500.00000,70,1,0
2,3,6,2,47880.0,2012-04-01,5.0,3,1,2023-08-31,11,renewal,142968.91392,136,1,0
3,4,31,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0
4,5,32,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0
5,6,33,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0
6,7,34,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0
7,8,36,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0
8,9,37,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0
9,37,46,2,NaN,2013-10-01,5.0,3,1,2023-08-31,9,renewal,NaN,118,1,0
10,39,51,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0


In [18]:
#
# Multiply the amount and factor to get rent
df['Rent'] = df['revised_amount'] * df['factor']
#
#Print the dataframe
df


,agreement,room,client,amount,start_date,duration,review,quarterly,rpt_start,age,phase,revised_amount,month_diff,mod,factor,Rent
1,2,5,2,5500.0,2017-10-01,5.0,3,1,2023-08-31,5,renewal,5500.00000,70,1,0,0.000
2,3,6,2,47880.0,2012-04-01,5.0,3,1,2023-08-31,11,renewal,142968.91392,136,1,0,0.000
3,4,31,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0,0.000
4,5,32,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0,0.000
5,6,33,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0,0.000
6,7,34,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0,0.000
7,8,36,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0,0.000
8,9,37,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0,0.000
9,37,46,2,NaN,2013-10-01,5.0,3,1,2023-08-31,9,renewal,NaN,118,1,0,NaN
10,39,51,2,5000.0,2013-10-01,5.0,3,1,2023-08-31,9,renewal,10368.00000,118,1,0,0.000
